In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 15.8 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 60.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
import json
import os

import torch.nn as nn
import torch
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

### Change the path here

In [4]:
training_filepath = './drive/MyDrive/BioASQ/BioASQ-training7b/trainining7b.json'
test_directory = './drive/MyDrive/BioASQ/Task7BGoldenEnriched/'

In [5]:
training_text = []
test_text = []

# Filter out the training data
with open (training_filepath, "r") as f:
    data = json.loads(f.read())
    texts = data['questions']
    for text in texts:
        if 'exact_answer' in text.keys():
            if text['exact_answer'] == 'yes' or text['exact_answer'] == 'no':
                training_text.append(text)

# Filter out the text data
directory = test_directory
for filename in os.listdir(directory):
    with open (directory+filename, "r") as f:
        data = json.loads(f.read())
        texts = data['questions']
        for text in texts:
            if 'exact_answer' in text.keys():
                if text['exact_answer'] == 'yes' or text['exact_answer'] == 'no':
                    test_text.append(text)

In [6]:
def process_data(texts):
    question_list = []
    context_list = []
    target_list = []
    for text in texts:
        question_list.append(text['body'])
        context_list.append(' '. join([x['text'] for x in text['snippets']]))
        target_list.append(1 if text['exact_answer'] == 'yes' else 0)
    df = pd.DataFrame(zip(question_list, context_list, target_list), columns=['question', 'context', 'target'])
    return df

In [7]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.out = nn.Linear(768, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.out(output)
        return self.softmax(output)

class BERTDatasetTraining:
    def __init__(self, question, context, target, tokenizer, max_len):
        self.question = question
        self.context = context
        self.target = target

        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.question)

    def __getitem__(self, item):
        question= str(self.question[item])
        context = str(self.context[item])

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True
        )

        ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        padding_len = self.max_len - len(ids)

        ids = ids[:self.max_len] + ([0] * padding_len)
        token_type_ids = token_type_ids[:self.max_len] + ([0] * padding_len)
        mask = mask[:self.max_len] + ([0] * padding_len)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[item], dtype=torch.long)
        }


def loss_fn(outputs, target):
    return nn.CrossEntropyLoss()(outputs, target)


def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d['ids']
        mask = d['mask']
        token_type_ids = d['token_type_ids']
        target = d['target']

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.long)

        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, target)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi % 10 == 0:
            print(f'bi={bi}, loss={loss}')


def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        with torch.no_grad():
            ids = d['ids'].to(device, dtype=torch.long)
            mask = d['mask'].to(device, dtype=torch.long)
            token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
            target = d['target'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)

            fin_targets.append(target.cpu().detach().numpy())
            fin_outputs.append(outputs.cpu().detach().numpy())

    return np.vstack(fin_outputs), np.hstack(fin_targets)

In [10]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 4
EPOCHS = 8
LEARNING_RATE = 3e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = BERTBaseUncased().to(device)

train_df = process_data(training_text)
test_df = process_data(test_text)

train_dataset = BERTDatasetTraining(
    question=train_df.question.values,
    context=train_df.context.values,
    target=train_df.target.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)

test_dataset = BERTDatasetTraining(
    question=test_df.question.values,
    context=test_df.context.values,
    target=test_df.target.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=True,
    drop_last=True
)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE * EPOCHS)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

for epoch in range(EPOCHS):
    train_loop_fn(train_data_loader, model, optimizer, device, scheduler)
    output, target = eval_loop_fn(test_data_loader, model, device)
    acc = (output.argmax(1) == target).sum() / len(target)
    print(f'epoch: {epoch}, acc: {acc}')
    print(output.tolist())
    print(target.tolist())
    print('---------------------')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

bi=0, loss=0.7385053634643555
bi=10, loss=0.5747361183166504
bi=20, loss=0.5611595511436462
bi=30, loss=0.4443320631980896
bi=40, loss=0.44367051124572754
bi=50, loss=0.5627385377883911
bi=60, loss=0.5655116438865662
bi=70, loss=0.31829461455345154
bi=80, loss=0.3170015811920166
bi=90, loss=0.31648385524749756
epoch: 0, acc: 0.6714285714285714
[[0.031207473948597908, 0.9687925577163696], [0.005111492704600096, 0.9948884844779968], [0.004598370753228664, 0.9954016208648682], [0.006728799548000097, 0.9932711720466614], [0.004977671429514885, 0.9950222969055176], [0.005118867848068476, 0.9948811531066895], [0.004791452549397945, 0.9952085614204407], [0.006561230402439833, 0.9934387803077698], [0.004714139271527529, 0.9952859282493591], [0.006792582105845213, 0.9932073950767517], [0.004782333970069885, 0.9952176809310913], [0.005505098029971123, 0.9944948554039001], [0.006798095069825649, 0.9932019114494324], [0.005028888583183289, 0.9949710369110107], [0.006148815155029297, 0.993851184844